# experiment1

In [4]:
import pymc as pm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
import warnings

# Ignore FutureWarning
warnings.filterwarnings('ignore', category=FutureWarning)
# Ignore UserWarning
warnings.filterwarnings('ignore', category=UserWarning)

# Set matplotlib and seaborn configuration
plt.rcParams['font.sans-serif'] = ['SimHei']  # Use SimHei font (Chinese characters)
plt.rcParams['axes.unicode_minus'] = False    # Solve the issue of displaying negative signs
sns.set_style("white")

In [2]:
s_num = 100
with pm.Model() as model:
    # Root node C
    C = pm.Categorical('C', p=np.array([0.3, 0.7]))

    # Define mean and standard deviation based on the value of C
    mean_S = pm.math.switch(pm.math.eq(C, 0), 620, 1600)
    std_S = pm.math.switch(pm.math.eq(C, 0), 200, 300)  # Standard deviation changes based on the value of C

    # Define S1 to S100
    S_list = []
    for i in range(1, s_num + 1):
        Si = pm.Normal(f'S{i}', mu=mean_S, sigma=std_S)  # Use mean and standard deviation defined by C
        S_list.append(Si)

    # Define T5, T10, T15, etc., ensuring that T_i is independent from T_{i-1}
    T_list = []
    for i in range(5, s_num + 1, 5):
        if i == 5:
            # T5 = S1 + S2 + ... + S5
            Ti = pm.Deterministic(f'T{i}', sum(S_list[:i]))
        else:
            # Copy the value of T_{i-5} to ensure independence
            T_prev_copy = pm.Deterministic(f'T{i-5}_copy', T_list[-1])
            # T10 = T_prev_copy + S6 + S7 + ... + S10
            Ti = pm.Deterministic(f'T{i}', T_prev_copy + sum(S_list[i-5:i]))
        T_list.append(Ti)
        
    # Define the CPT for F
    p_F_0 = np.array([
        9.950248756218905E-4, 9.950248756218905E-4, 9.950248756218905E-4, 9.950248756218905E-4,
        9.950248756218905E-4, 9.950248756218905E-4, 9.950248756218905E-4, 9.950248756218905E-4,
        0.006965174129353234, 0.01691542288557214, 0.04378109452736319, 0.08258706467661692,
        0.1721393034825871, 0.1870646766169154, 0.1960199004975124, 0.1422885572139304,
        0.08656716417910448, 0.03482587064676617, 0.01791044776119403, 0.004975124378109453
    ])  # When C = 0

    p_F_1 = np.array([
        0.00298804780876494, 0.01195219123505976, 0.04780876494023904, 0.0796812749003984,
        0.1464143426294821, 0.1852589641434263, 0.198207171314741, 0.149402390438247,
        0.1115537848605578, 0.03685258964143426, 0.01593625498007968, 0.00597609561752988,
        9.9601593625498E-4, 9.9601593625498E-4, 9.9601593625498E-4, 9.9601593625498E-4,
        9.9601593625498E-4, 9.9601593625498E-4, 9.9601593625498E-4, 9.9601593625498E-4
    ])  # When C = 1

    # Use pm.math.switch to choose p_F based on the value of C
    p_F = pm.math.switch(pm.math.eq(C, 0), p_F_0, p_F_1)

    # Define F
    F = pm.Categorical('F', p=p_F)

    # Convert T_list to a tensor
    T_array = pm.math.stack(T_list)

    # Ensure F is an integer index
    F_index = pm.intX(F)

    # Select the corresponding T_i based on the value of F
    T = pm.Deterministic('T', T_array[F_index])

    # Sample from the prior distribution
    prior = pm.sample_prior_predictive(samples=10000)

# Print the prior sampling results
print(prior)

Sampling: [C, F, S1, S10, S100, S11, S12, S13, S14, S15, S16, S17, S18, S19, S2, S20, S21, S22, S23, S24, S25, S26, S27, S28, S29, S3, S30, S31, S32, S33, S34, S35, S36, S37, S38, S39, S4, S40, S41, S42, S43, S44, S45, S46, S47, S48, S49, S5, S50, S51, S52, S53, S54, S55, S56, S57, S58, S59, S6, S60, S61, S62, S63, S64, S65, S66, S67, S68, S69, S7, S70, S71, S72, S73, S74, S75, S76, S77, S78, S79, S8, S80, S81, S82, S83, S84, S85, S86, S87, S88, S89, S9, S90, S91, S92, S93, S94, S95, S96, S97, S98, S99]


Inference data with groups:
	> prior


In [3]:
# Assume extracting data from the existing `prior` 
prior_dataset = prior['prior']  # `prior` contains the sampling results from the model

# Check the keys of the variables
print(prior_dataset.keys())

KeysView(<xarray.Dataset>
Dimensions:   (chain: 1, draw: 10000)
Coordinates:
  * chain     (chain) int32 0
  * draw      (draw) int32 0 1 2 3 4 5 6 ... 9993 9994 9995 9996 9997 9998 9999
Data variables: (12/142)
    C         (chain, draw) int64 0 1 1 1 1 1 0 0 1 1 1 ... 0 1 1 0 0 0 1 1 1 1
    F         (chain, draw) int64 16 5 8 4 5 2 11 12 4 ... 5 5 11 15 14 3 6 7 4
    S1        (chain, draw) float64 668.4 1.553e+03 ... 2.2e+03 1.582e+03
    S10       (chain, draw) float64 723.0 1.256e+03 ... 758.0 1.256e+03
    S100      (chain, draw) float64 409.3 1.743e+03 ... 1.914e+03 1.797e+03
    S11       (chain, draw) float64 547.4 1.467e+03 ... 1.443e+03 1.717e+03
    ...        ...
    T85       (chain, draw) float64 5.642e+04 1.36e+05 ... 1.381e+05 1.357e+05
    T85_copy  (chain, draw) float64 5.642e+04 1.36e+05 ... 1.381e+05 1.357e+05
    T90       (chain, draw) float64 5.917e+04 1.441e+05 ... 1.461e+05 1.442e+05
    T90_copy  (chain, draw) float64 5.917e+04 1.441e+05 ... 1.461e+05 1.4

In [5]:
# Extract sample data
C_samples = prior_dataset['C'].values.flatten()  # Flatten the samples of variable C
F_samples = prior_dataset['F'].values.flatten()  # Flatten the samples of variable F
T85_samples = prior_dataset['T85'].values.flatten()  # Flatten the samples of variable T85
T_samples = prior_dataset['T'].values.flatten()  # Flatten the samples of variable T

In [7]:
# Extract sample data for T
T_samples = prior_dataset['T'].values.flatten()

# Create a Plotly chart
fig = go.Figure()

# Add a histogram
fig.add_trace(go.Histogram(
    x=T_samples,
    histnorm='density',  # Normalize to density
    name='Histogram',
    marker_color='black',
    opacity=0.6
))

# Update layout
fig.update_layout(
    title='Distribution of T',
    xaxis_title='T',
    yaxis_title='Density',
    bargap=0.2,  # Gap between histogram bars
    width=600,    # Set chart width to 800 pixels
    height=500,   # Optional: Set chart height
    template='plotly_white'  # Use white template
)

# Display the chart
fig.show()

In [8]:
# Define the function to calculate P(C|T=target_value)
def calculate_conditional_probability(T_samples, C_samples, target_T_value, threshold=2):
    # Find samples of T close to target_T_value
    condition_indices = np.abs(T_samples - target_T_value) < threshold
    
    # Get the distribution of C for these samples
    C_given_T = C_samples[condition_indices]
    
    # Calculate P(C=0 | T=target_value) and P(C=1 | T=target_value)
    P_C_given_T_0 = np.mean(C_given_T == 0) if len(C_given_T) > 0 else 0
    P_C_given_T_1 = np.mean(C_given_T == 1) if len(C_given_T) > 0 else 0
    
    # Return conditional probabilities
    return P_C_given_T_0, P_C_given_T_1


# Define 10 observation values
evidence_values = [500, 10000, 30000, 40000, 50000, 60000, 70000, 80000, 120000, 160000]
threshold = 1000  # Given threshold

# Store results
results = []

# Calculate conditional probability for each observation value
for value in evidence_values:
    P_C_0, P_C_1 = calculate_conditional_probability(T_samples, C_samples, target_T_value=value, threshold=threshold)
    results.append({'Target_T_Value': value, 'P(C=0|T)': P_C_0, 'P(C=1|T)': P_C_1})

# Convert results to DataFrame
df = pd.DataFrame(results)
df

Target_T_Value  P(C=0|T)  P(C=1|T)
0             500  0.000000  0.000000
1           10000  0.500000  0.500000
2           30000  0.172840  0.827160
3           40000  0.364428  0.635572
4           50000  0.475410  0.524590
5           60000  0.181102  0.818898
6           70000  0.000000  1.000000
7           80000  0.000000  1.000000
8          120000  0.000000  1.000000
9          160000  0.000000  1.000000

# experiment2

In [9]:
s_num = 100

# CPTs
C1_cpt = np.array([
    [0.09090909090909091, 0.2727272727272727, 0.6363636363636364],  # C = 0
    [0.2, 0.5, 0.3],  # C = 1
    [0.5625, 0.3125, 0.125]   # C = 2
])

C2_cpt = np.array([
    [0.02857142857142857, 0.05714285714285714, 0.9142857142857143],  # C = 0
    [0.037734632602485, 0.9433974190171714, 0.01886794838034356],  # C = 1
    [0.2962962962962963, 0.6831275720164609, 0.02057613168724282]   # C = 2
])

C3_cpt = np.array([
    [0.007299270072992701, 0.145985401459854, 0.8467153284671532],  # C = 0
    [0.12, 0.6799999999999999, 0.2000000000000001],  # 当 C = 1
    [0.9411764705882353, 0.05882352941176471, 0]   # C = 2
])


C4_cpt = np.array([
    [0.2, 0.3, 0.5],  # C = 0
    [0.2, 0.7, 0.1],  # C = 1
    [0.9, 0.1, 0  ]   # C = 2
])

C5_cpt = np.array([
    [0.1, 0.2, 0.7],  # C = 0
    [0.6666666666666666, 0.2, 0.1333333333333334],                # C = 1
    [0.1, 0.9 ,0]   # C = 2
])

C6_cpt = np.array([
    [0.08620689655172414, 0.8620689655172414, 0.05172413793103448],  # C = 0
    [0.06896551724137931, 0.896551724137931, 0.03448275862068961],     # C = 1
    [0.2, 0.1, 0.7]   # C = 2
])

C7_cpt = np.array([
    [0.144927536231884, 0.00966183574879227, 0.8454106280193237],  # C = 0
    [0.04651162790697675, 0.9302325581395349, 0.02325581395348841],  # C = 1
    [0.06410256410256411, 0.2948717948717949, 0.641025641025641]    # C = 2
])

C9_cpt_dict = {
    (0, 0): [0.5, 0.4, 0.1],
    (0, 1): [0.01162790697674419, 0.813953488372093, 0.1744186046511628],
    (0, 2): [0.2, 0.3, 0.5],
    (1, 0): [0.1, 0.2, 0.7],
    (1, 1): [0.8928571428571429, 0.08928571428571429, 0.01785714285714279],
    (1, 2): [0.0, 0.8, 0.2],
    (2, 0): [0.1, 0.2, 0.7],
    (2, 1): [0.2, 0.8, 0.0],
    (2, 2): [0.2, 0.3, 0.5]
}

C10_cpt_dict = {
    (0, 0): [0.1, 0.9],
    (0, 1): [0.9, 0.1],
    (0, 2): [0.6, 0.4],
    (1, 0): [0.2, 0.8],
    (1, 1): [0.7, 0.3],
    (1, 2): [0.9, 0.1],
    (2, 0): [0.6, 0.4],
    (2, 1): [0.7, 0.3],
    (2, 2): [0.4, 0.6]
}


p_F_0 = np.array([
    0.000992063492063492, 0.000992063492063492, 0.000992063492063492,
    0.000992063492063492, 0.000992063492063492, 0.000992063492063492,
    0.000992063492063492, 0.01091269841269841, 0.03869047619047619,
    0.08432539682539683, 0.1636904761904762, 0.2053571428571428,
    0.2063492063492063, 0.1507936507936508, 0.08134920634920635,
    0.03869047619047619, 0.008928571428571428, 0.001984126984126984,
    0.000992063492063492, 0.000992063492063492
])  # C6 = 0

p_F_1 = np.array([
    0.0009881422924901185, 0.0009881422924901185, 0.002964426877470356,
    0.04347826086956522, 0.1482213438735178, 0.2875494071146245,
    0.2994071146245059, 0.1442687747035573, 0.0533596837944664,
    0.008893280632411068, 0.0009881422924901185, 0.0009881422924901185,
    0.0009881422924901185, 0.0009881422924901185, 0.0009881422924901185,
    0.0009881422924901185, 0.0009881422924901185, 0.0009881422924901185,
    0.0009881422924901185, 0.0009881422924901185
])  # C6 = 1

p_F_2 = np.array([
    0.0009930486593843098, 0.0009930486593843098, 0.0009930486593843098,
    0.01092353525322741, 0.03177755710029791, 0.07845084409136048,
    0.1400198609731877, 0.2204568023833168, 0.2393247269116187,
    0.1737835153922542, 0.06852035749751738, 0.02184707050645482,
    0.003972194637537239, 0.00198609731876862, 0.0009930486593843098,
    0.0009930486593843098, 0.0009930486593843098, 0.0009930486593843098,
    0.0009930486593843098, 0.0009930486593843098
])  # C6 = 2

In [10]:
with pm.Model() as model:
    # Root node C
    C = pm.Categorical('C', p=np.array([0.5, 0.2, 0.3]))
    
    # Define C1
    p_C1 = pm.math.switch(pm.math.eq(C, 0), C1_cpt[0],
                          pm.math.switch(pm.math.eq(C, 1), C1_cpt[1], C1_cpt[2]))
    C1 = pm.Categorical('C1', p=p_C1)
    
    # Define C2
    p_C2 = pm.math.switch(pm.math.eq(C, 0), C2_cpt[0],
                          pm.math.switch(pm.math.eq(C, 1), C2_cpt[1], C2_cpt[2]))
    C2 = pm.Categorical('C2', p=p_C2)
    
    # Define C3
    p_C3 = pm.math.switch(pm.math.eq(C, 0), C3_cpt[0],
                          pm.math.switch(pm.math.eq(C, 1), C3_cpt[1], C3_cpt[2]))
    C3 = pm.Categorical('C3', p=p_C3)
    
    # Define C4
    p_C4 = pm.math.switch(pm.math.eq(C1, 0), C4_cpt[0],
                          pm.math.switch(pm.math.eq(C1, 1), C4_cpt[1], C4_cpt[2]))
    C4 = pm.Categorical('C4', p=p_C4)
    
    # Define C5
    p_C5 = pm.math.switch(pm.math.eq(C2, 0), C5_cpt[0],
                          pm.math.switch(pm.math.eq(C2, 1), C5_cpt[1], C5_cpt[2]))
    C5 = pm.Categorical('C5', p=p_C5)
    
    # Define C6
    p_C6 = pm.math.switch(pm.math.eq(C2, 0), C6_cpt[0],
                          pm.math.switch(pm.math.eq(C2, 1), C6_cpt[1], C6_cpt[2]))
    C6 = pm.Categorical('C6', p=p_C6)
    
    # Define C7
    p_C7 = pm.math.switch(pm.math.eq(C3, 0), C7_cpt[0],
                          pm.math.switch(pm.math.eq(C3, 1), C7_cpt[1], C7_cpt[2]))
    C7 = pm.Categorical('C7', p=p_C7)
    
    # Define C9, using pm.math.switch to select probabilities based on the states of C5 and C6
    p_C9 = pm.math.switch(
        pm.math.eq(C5, 0) & pm.math.eq(C6, 0), C9_cpt_dict[(0, 0)],
        pm.math.switch(
            pm.math.eq(C5, 0) & pm.math.eq(C6, 1), C9_cpt_dict[(0, 1)],
            pm.math.switch(
                pm.math.eq(C5, 0) & pm.math.eq(C6, 2), C9_cpt_dict[(0, 2)],
                pm.math.switch(
                    pm.math.eq(C5, 1) & pm.math.eq(C6, 0), C9_cpt_dict[(1, 0)],
                    pm.math.switch(
                        pm.math.eq(C5, 1) & pm.math.eq(C6, 1), C9_cpt_dict[(1, 1)],
                        pm.math.switch(
                            pm.math.eq(C5, 1) & pm.math.eq(C6, 2), C9_cpt_dict[(1, 2)],
                            pm.math.switch(
                                pm.math.eq(C5, 2) & pm.math.eq(C6, 0), C9_cpt_dict[(2, 0)],
                                pm.math.switch(
                                    pm.math.eq(C5, 2) & pm.math.eq(C6, 1), C9_cpt_dict[(2, 1)],
                                    C9_cpt_dict[(2, 2)]
                                )
                            )
                        )
                    )
                )
            )
        )
    )
    C9 = pm.Categorical('C9', p=p_C9)
    
    # Define C10, using pm.math.switch to select probabilities based on the states of C6 and C7
    p_C10 = pm.math.switch(
        pm.math.eq(C6, 0) & pm.math.eq(C7, 0), C10_cpt_dict[(0, 0)],
        pm.math.switch(
            pm.math.eq(C6, 0) & pm.math.eq(C7, 1), C10_cpt_dict[(0, 1)],
            pm.math.switch(
                pm.math.eq(C6, 0) & pm.math.eq(C7, 2), C10_cpt_dict[(0, 2)],
                pm.math.switch(
                    pm.math.eq(C6, 1) & pm.math.eq(C7, 0), C10_cpt_dict[(1, 0)],
                    pm.math.switch(
                        pm.math.eq(C6, 1) & pm.math.eq(C7, 1), C10_cpt_dict[(1, 1)],
                        pm.math.switch(
                            pm.math.eq(C6, 1) & pm.math.eq(C7, 2), C10_cpt_dict[(1, 2)],
                            pm.math.switch(
                                pm.math.eq(C6, 2) & pm.math.eq(C7, 0), C10_cpt_dict[(2, 0)],
                                pm.math.switch(
                                    pm.math.eq(C6, 2) & pm.math.eq(C7, 1), C10_cpt_dict[(2, 1)],
                                    C10_cpt_dict[(2, 2)]
                                )
                            )
                        )
                    )
                )
            )
        )
    )
    C10 = pm.Categorical('C10', p=p_C10)

    # Define C8: Select different Gaussian distributions based on the state of C4
    mu_C8 = pm.math.switch(pm.math.eq(C4, 0), 2000,
                           pm.math.switch(pm.math.eq(C4, 1), 300, 1000))
    sigma_C8 = pm.math.switch(pm.math.eq(C4, 0), 200,
                              pm.math.switch(pm.math.eq(C4, 1), 50, 100))
    
    # Define C8 as a Gaussian distribution
    C8 = pm.Normal('C8', mu=mu_C8, sigma=sigma_C8)
    
    # Create 100 independent nodes S1, S2, ..., S100 in bulk
    S_list = []
    for i in range(100):
        # Define the value of each S node based on the state of C10
        S = pm.Deterministic(f'S{i+1}', 
                             pm.math.switch(pm.math.eq(C10, 0), 0.2 * C8, 0.8 * C8))
        S_list.append(S)

    # Define T5, T10, T15, etc., ensuring that T_i is independent of T_{i-1}
    T_list = []
    for i in range(5, s_num + 1, 5):
        if i == 5:
            # T5 = S1 + S2 + ... + S5
            Ti = pm.Deterministic(f'T{i}', pm.math.sum(S_list[:i]))  # Use pm.math.sum
        else:
            # Create a copy of T_{i-5} to ensure independence
            T_prev_copy = pm.Deterministic(f'T{i-5}_copy', T_list[-1])
            # T{i} = T_prev_copy + S_{i-4} + S_{i-3} + S_{i-2} + S_{i-1} + S_i
            Ti = pm.Deterministic(f'T{i}', T_prev_copy + pm.math.sum(S_list[i-5:i]))  # Use pm.math.sum
        T_list.append(Ti)

    # Use nested pm.math.switch to select p_F based on the value of C9
    p_F = pm.math.switch(pm.math.eq(C9, 0), p_F_0,
                         pm.math.switch(pm.math.eq(C9, 1), p_F_1, p_F_2))
    
    # Define F, based on the selected p_F as the probability distribution
    F = pm.Categorical('F', p=p_F)

    # Convert T_list into a tensor
    T_array = pm.math.stack(T_list)

    # Ensure F is an integer index
    F_index = pm.intX(F)

    # Select the corresponding T_i based on the value of F
    T = pm.Deterministic('T', T_array[F_index])

    # Sample from the prior distribution and check the results
    prior = pm.sample_prior_predictive(samples=10000)

# Print the prior sampling results
print(prior)

Sampling: [C, C1, C10, C2, C3, C4, C5, C6, C7, C8, C9, F]


Inference data with groups:
	> prior


In [11]:
# Assume extracting data from the existing `prior` 
prior_dataset = prior['prior']  # `prior` contains the sampling results from the model

# Check the keys of the variables
print(prior_dataset.keys())

KeysView(<xarray.Dataset>
Dimensions:   (chain: 1, draw: 10000)
Coordinates:
  * chain     (chain) int32 0
  * draw      (draw) int32 0 1 2 3 4 5 6 ... 9993 9994 9995 9996 9997 9998 9999
Data variables: (12/152)
    C         (chain, draw) int64 0 0 2 0 1 1 0 0 0 2 2 ... 0 1 1 0 0 2 0 0 0 0
    C1        (chain, draw) int64 0 2 0 2 1 1 2 2 2 0 0 ... 2 2 1 1 1 0 2 2 2 2
    C10       (chain, draw) int64 0 1 0 0 0 0 1 1 0 0 0 ... 1 1 1 0 1 0 1 0 1 1
    C2        (chain, draw) int64 2 2 1 2 1 1 2 2 2 0 0 ... 2 1 1 2 2 1 2 2 2 2
    C3        (chain, draw) int64 2 2 0 2 2 2 2 2 2 0 0 ... 2 1 1 2 2 0 1 2 2 2
    C4        (chain, draw) int64 2 0 1 0 1 1 0 0 0 2 2 ... 0 1 0 0 1 2 0 0 0 0
    ...        ...
    T85       (chain, draw) float64 1.854e+04 1.383e+05 ... 1.544e+05 1.282e+05
    T85_copy  (chain, draw) float64 1.854e+04 1.383e+05 ... 1.544e+05 1.282e+05
    T90       (chain, draw) float64 1.963e+04 1.464e+05 ... 1.635e+05 1.357e+05
    T90_copy  (chain, draw) float64 1.963e+04 1.4

In [13]:
# Extract sample data for T
T_samples = prior_dataset['T'].values.flatten()

# Create a Plotly chart
fig = go.Figure()

# Add a histogram
fig.add_trace(go.Histogram(
    x=T_samples,
    histnorm='density',  # Normalize to density
    name='Histogram',
    marker_color='black',
    opacity=0.6
))

# Update layout
fig.update_layout(
    title='Distribution of T',
    xaxis_title='T',
    yaxis_title='Density',
    bargap=0.2,  # Gap between histogram bars
    width=600,    # Set chart width to 800 pixels
    height=500,   # Optional: Set chart height
    template='plotly_white'  # Use white template
)

# Display the chart
fig.show()

In [17]:
def calculate_conditional_probability(T_samples, C_samples, target_T_value, threshold=2):
    """Calculate P(C | T=target value)"""
    condition_indices = np.abs(T_samples - target_T_value) < threshold
    C_given_T = C_samples[condition_indices]
    unique_states = np.unique(C_samples)
    P_C_given_T = {state: 0 for state in unique_states}
    unique_given_T, counts_given_T = np.unique(C_given_T, return_counts=True)
    total_counts = counts_given_T.sum()
    for state, count in zip(unique_given_T, counts_given_T):
        P_C_given_T[state] = count / total_counts if total_counts > 0 else 0
    return P_C_given_T

def calculate_conditional_probabilities(T_samples, prior_dataset, discrete_node_names, num_states_dict, target_T_values, threshold=1000):
    """Calculate the conditional probabilities for each T value."""
    results = []

    for target_T_value in target_T_values:
        row_data = {'T_value': target_T_value}  # Data for each observation value

        # Calculate the conditional probability of each discrete node
        for node in discrete_node_names:
            C_samples = prior_dataset[node].values.flatten()
            num_states = num_states_dict[node]
            conditional_probs = calculate_conditional_probability(T_samples, C_samples, target_T_value, threshold)
            for state in range(num_states):
                row_data[f'{node}_state{state}_prob'] = conditional_probs.get(state, 0)

        # Add this row of data to the results list
        results.append(row_data)

    # Convert the results to a DataFrame
    results_df = pd.DataFrame(results)
    return results_df

# Define the names of discrete nodes and their number of states
discrete_node_names = ['C', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C9', 'C10']
num_states_dict = {
    'C': 3, 
    'C1': 3,  
    'C2': 3, 
    'C3': 3,  
    'C4': 3,  
    'C5': 3,
    'C6': 3,  
    'C7': 3,  
    'C9': 3,
    'C10': 2,
}

# List of given T values
target_T_values = [1000, 8000, 15000, 25000, 35000, 50000, 70000, 90000, 110000, 140000]

# Extract T samples
T_samples = prior_dataset['T'].values.flatten()

# Calculate the conditional probabilities for the discrete nodes and generate the table
conditional_probabilities_df = calculate_conditional_probabilities(T_samples, prior_dataset, discrete_node_names, num_states_dict, target_T_values, threshold=1000)

# Display the results
conditional_probabilities_df.tail(55)

T_value  C_state0_prob  C_state1_prob  C_state2_prob  C1_state0_prob  \
0     1000       0.336303       0.289532       0.374165        0.210468   
1     8000       0.456982       0.194640       0.348378        0.375176   
2    15000       0.588542       0.168403       0.243056        0.184028   
3    25000       0.426036       0.201183       0.372781        0.378698   
4    35000       0.602941       0.117647       0.279412        0.308824   
5    50000       0.703704       0.157407       0.138889        0.092593   
6    70000       0.875000       0.093750       0.031250        0.093750   
7    90000       0.692308       0.076923       0.230769        0.076923   
8   110000       0.500000       0.375000       0.125000        0.125000   
9   140000       0.000000       0.000000       1.000000        0.000000   

   C1_state1_prob  C1_state2_prob  C2_state0_prob  C2_state1_prob  \
0        0.689310        0.100223        0.128062        0.567929   
1        0.416079        0.208745        0.107193        0.458392   
2        0.187500        0.628472        0.083333        0.375000   
3        0.136095        0.485207        0.136095        0.461538   
4        0.147059        0.544118        0.176471        0.220588   
5        0.166667        0.740741        0.055556        0.259259   
6        0.062500        0.843750        0.062500        0.093750   
7        0.153846        0.769231        0.230769        0.153846   
8        0.125000        0.750000        0.000000        0.500000   
9        1.000000        0.000000        0.000000        1.000000   

   C2_state2_prob  ...  C6_state1_prob  C6_state2_prob  C7_state0_prob  \
0        0.304009  ...        0.662584        0.255011        0.042316   
1        0.434415  ...        0.526093        0.373766        0.094499   
2        0.541667  ...        0.446181        0.373264        0.059028   
3        0.402367  ...        0.715976        0.136095        0.136095   
4        0.602941  ...        0.426471        0.397059        0.102941   
5        0.685185  ...        0.287037        0.620370        0.138889   
6        0.843750  ...        0.125000        0.625000        0.093750   
7        0.615385  ...        0.384615        0.307692        0.307692   
8        0.500000  ...        0.625000        0.125000        0.250000   
9        0.000000  ...        1.000000        0.000000        1.000000   

   C7_state1_prob  C7_state2_prob  C9_state0_prob  C9_state1_prob  \
0        0.395323        0.562361        0.007795        0.888641   
1        0.344147        0.561354        0.025388        0.784203   
2        0.394097        0.546875        0.135417        0.614583   
3        0.355030        0.508876        0.532544        0.355030   
4        0.264706        0.632353        0.308824        0.485294   
5        0.277778        0.583333        0.055556        0.842593   
6        0.250000        0.656250        0.062500        0.343750   
7        0.000000        0.692308        0.615385        0.000000   
8        0.625000        0.125000        0.875000        0.000000   
9        0.000000        0.000000        1.000000        0.000000   

   C9_state2_prob  C10_state0_prob  C10_state1_prob  
0        0.103563         0.998886         0.001114  
1        0.190409         0.538787         0.461213  
2        0.250000         0.904514         0.095486  
3        0.112426         0.633136         0.366864  
4        0.205882         0.308824         0.691176  
5        0.101852         0.000000         1.000000  
6        0.593750         0.000000         1.000000  
7        0.384615         0.000000         1.000000  
8        0.125000         0.000000         1.000000  
9        0.000000         0.000000         1.000000  

[10 rows x 30 columns]